In [1]:
import pandas as pd
import numpy as np

<h2>Trading Strategy</h2>

Now that we have the predicted price for a given time period, we can build a portfolio at each time step so that we can have an idea how much profit can be generated by using the SMA prediction model. The strategy employed to build the portfolio will be standardized accross all the different prediction model we will explore later as well.

The strategy will be to invest in the top $n$ coins with the highest expected ROI and then sell it the following day. To simplify the calculation, we will be ignoring the transaction cost.

We will be trading for 1 year from April 24, 2017 and April 24, 2018.

In [2]:
def update_mean(mean, t, new_value):
    if t == 0:
        return new_value
    else:
        return (mean * (t - 1) + new_value) / t

# Function to update standard deviation based on new value
def update_std(std, mean, new_mean, t, new_value):
    if t == 0:
        return 0
    else:
        return np.sqrt((std ** 2 * (t - 1) + (new_value - new_mean) * (new_value - mean)) / t)

In [35]:
import pandas as pd
import numpy as np

def update_mean(mean, t, new_value):
    if t == 0:
        return new_value
    else:
        return (mean * (t - 1) + new_value) / t

# Function to update standard deviation based on new value
def update_std(std, mean, new_mean, t, new_value):
    if t == 0:
        return 0
    else:
        return np.sqrt((std ** 2 * (t - 1) + (new_value - new_mean) * (new_value - mean)) / t)

# Function to execute the trading strategy over the trading horizon using asset matrix
def evaluate(result, target, n=10, initial_value=10000):
    total_value = initial_value
    mean_roi = 0
    std_roi = 0

    dates = list(set(result.time))
    dates.sort()
    
    history = {}
    
    df = result.copy()
    if target == 'price':
        df['predicted_roi'] = (df['pred']/df['price_lag_1']) - 1
    else:
        df['predicted_roi'] = df['pred']
    df.sort_values(by='predicted_roi', ascending=False, inplace=True)
    
    t = 1

    for date in dates:
        temp_df = df.query('time == @date & predicted_roi > 0')

        if not temp_df.empty:
            top_n = temp_df.nlargest(n, 'predicted_roi')
            selected_n = len(top_n)
            currencies = list(top_n.sym)
            day_return = sum(top_n['roi'] * total_value / selected_n)
            day_roi = day_return/total_value
        else:
            currencies = []
            day_return = 0
            day_roi = 0
        
        total_value += day_return
        percent_returns = (total_value/initial_value - 1) * 100

        prev_mean_roi = mean_roi
        mean_roi = update_mean(prev_mean_roi, t, day_roi)
        std_roi = update_std(std_roi, prev_mean_roi, mean_roi, t, day_roi)
        std_roi = 0.001 if (std_roi == 0) else std_roi
        sharpe_ratio = mean_roi/std_roi
        
        history[date] = [total_value, day_roi, currencies, percent_returns, sharpe_ratio]

        t += 1
        
    print('Cumulative Returns: {:.2e}%, Sharpe Ratio : {:.4e}' \
                          .format(percent_returns, sharpe_ratio))
    
    history = pd.DataFrame.from_dict(history, orient='index', 
                                     columns=['total', 'roi', 'currencies', '%return', 'sharpe'])
    
    return history

In [17]:
import source.load
from source.load import *
from source import create

data_dict = source.load.load_data()
data = data_dict['data']
symbols = data_dict['symbols']

In [22]:
features = create.create_features(data, W=3)

In [24]:
result = features['data']
result['pred'] = result['price_mean']

In [40]:
!pygmentize './source/strategy.py'

import pandas as pd
import numpy as np

def update_mean(mean, t, new_value):
    if t == 0:
        return new_value
    else:
        return (mean * (t - 1) + new_value) / t

# Function to update standard deviation based on new value
def update_std(std, mean, new_mean, t, new_value):
    if t == 0:
        return 0
    else:
        return np.sqrt((std ** 2 * (t - 1) + (new_value - new_mean) * (new_value - mean)) / t)

# Function to execute the trading strategy over the trading horizon using asset matrix
def evaluate(result, target, n=10, initial_value=10000):
    total_value = initial_value
    mean_roi = 0
    std_roi = 0

    dates = list(set(result.time))
    dates.sort()
    
    history = {}
    
    df = result.copy()
    if target == 'price':
        df['predicted_roi'] = (df['pred']/df['price_lag_1']) - 1
    else:
        df['predicted_roi'] = df['pred']
    df.sort_values(by='predicted_roi', ascending=False, inplace=True)
    
    t = 1

    for date in dates:
        temp_

In [38]:
from source import strategy
hist = strategy.evaluate(result, 'price')

Cumulative Returns: 2.39e+05%, Sharpe Ratio : 8.7976e-02


In [39]:
hist

,total,roi,currencies,%return,sharpe
2013-05-03,8.696254e+03,-0.130375,"[NMC, PPC, BTC, NVC, LTC]",-13.037459,-130.374588
2013-05-04,8.329574e+03,-0.042165,"[NMC, LTC, NVC, PPC, BTC]",-16.704263,-1.955778
2013-05-05,8.329574e+03,0.000000,[],-16.704263,-1.058720
2013-05-06,8.329574e+03,0.000000,[],-16.704263,-0.810343
2013-05-07,7.921508e+03,-0.048990,"[PPC, NMC, NVC, LTC, BTC]",-20.784925,-0.929463
...,...,...,...,...,...
2018-04-07,2.861218e+07,-0.108733,"[USNBT, RVR, QRK, ZET, BTCD, XMR, AMP, NMC, VR...",286021.785883,0.089249
2018-04-08,3.071150e+07,0.073372,"[MEC, QRK, XPM]",307014.971513,0.089684
2018-04-09,2.611880e+07,-0.149543,"[WDC, XPY, MEC]",261088.005018,0.088610
2018-04-10,2.702433e+07,0.034669,"[ZET, XPY, MEC, WDC, VTC, BTCD, XCP, DCR, NVC,...",270143.251702,0.088807
